In [36]:
import chess
import pandas as pd
import json

def load_game_data(file_name):
    file_path = f"{file_name}.json"
    with open(file_path, 'r') as file:
        games = json.load(file)
    return games

def classify_positions(game, middle_index):
    board = chess.Board()
    positions = []

    for move_number, san in enumerate(game['moves'].split(), start=1):
        try:
            move = board.parse_san(san)
            board.push(move)
            fen = board.fen()

            if move_number <= middle_index:
                game_state = "opening"
            else:
                game_state = "middlegame"

            positions.append({'position': fen, 'game_state': game_state})
        except Exception as e:
            print(f"Error processing move {san} in game {game['id']}: {e}")
            break  # or continue, if you want to skip the rest of the moves in this game

    return positions

def analyze_games(games):
    all_positions = []

    for game in games:
      #  middle_index = game['division'].get('middle', 25)
        middle_index = 25
        positions = classify_positions(game, middle_index)
        all_positions.extend(positions)

    return pd.DataFrame(all_positions)

In [37]:
import numpy as np
import chess

def fen_to_input(fen):
    board = chess.Board(fen)
    input_matrix = np.zeros((8, 8, 12), dtype=int)

    piece_to_plane = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

    for rank in range(8):
        for file in range(8):
            piece = board.piece_at(chess.square(file, 7 - rank))
            if piece:
                plane = piece_to_plane[piece.symbol()]
                input_matrix[rank, file, plane] = 1

    additional_planes = np.zeros((8, 8, 5), dtype=int)
    
    if board.has_kingside_castling_rights(chess.WHITE):
        additional_planes[:, :, 0] = 1
    if board.has_queenside_castling_rights(chess.WHITE):
        additional_planes[:, :, 1] = 1
    if board.has_kingside_castling_rights(chess.BLACK):
        additional_planes[:, :, 2] = 1
    if board.has_queenside_castling_rights(chess.BLACK):
        additional_planes[:, :, 3] = 1

    if board.ep_square:
        ep_rank = chess.square_rank(board.ep_square)
        ep_file = chess.square_file(board.ep_square)
        additional_planes[7 - ep_rank, ep_file, 4] = 1

    side_to_move_plane = np.ones((8, 8, 1), dtype=int) if board.turn == chess.WHITE else np.zeros((8, 8, 1), dtype=int)
    
    input_tensor = np.concatenate((input_matrix, additional_planes, side_to_move_plane), axis=2)
    
    return input_tensor

In [39]:

# Load games
games = load_game_data('apendra_games')

# Analyze games and get positions with classifications
results = analyze_games(games)

# Save results to JSON (optional)
#results.to_json("positions_classified.json", orient='records', lines=True)
# Remove duplicate positions
results = results.drop_duplicates(subset='position')

# Convert FEN to input tensor and add to DataFrame
results['input'] = results['position'].apply(lambda fen: fen_to_input(fen).flatten())

# Save results to JSON (optional# Save results to CSV
results.to_json("data/opening_classified.json")



Error processing move Bc4 in game 9HJKWvan: illegal san: 'Bc4' in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1


In [40]:
game_string = "e4 c6 Nc3 d5 f3 Nf6 d4 dxe4 fxe4 e6 Nf3 Nbd7 e5 Nd5 Nxd5 cxd5 Bb5 a6 Ba4 b5 Bb3 Bb7 a4 Be7 axb5 axb5 Rxa8 Qxa8 O-O O-O Bg5 Bxg5 Nxg5 h6 Nf3 f6 Qd3 fxe5 Nxe5 Nxe5 dxe5 Rxf1+ Kxf1 Qf8+ Kg1 Qf5 Qxb5 Bc8 Qe2 Bd7 h3 Qe4 Qxe4 dxe4 Kf2 Bc6 Bxe6+ Kf8 Bb3 Ke7 Ke3 g5 c3 Bb7 Bc2 Kd7 Bxe4 Ke6 Bxb7 Kxe5 Bf3 Kf5 Be2 Ke5 Kf2 Kf5 c4 Ke5 b4 Kd6 b5"

game = {
    'id': 'mock_game_1',
    'moves': game_string,
    'division': {'middle': 25}  # You can adjust the middle index as needed
}

# Wrap the game dictionary in a list
test_results = analyze_games([game])

# Remove duplicate positions
test_results = test_results.drop_duplicates(subset='position')

# Convert FEN to input tensor and add to DataFrame
test_results['input'] = test_results['position'].apply(lambda fen: fen_to_input(fen).flatten())

# Save the DataFrame to a CSV file

test_results.to_json("data/test.json")

